## Submitting Data to OpenAI via Batch

Once the data has been prepared and formmated, it is time to submit to OpenAI. There are several steps to consider in order to complete this process successfully.

- Uploading files
- Submitting files as batch submissions
- Checking batch status
- Collecting batch results

In order to get the ball rolling and give OpenAI access to our data, we upload our files to their API. With each upload, the API responds with some data about our upload, but most importantly it gives us a **File ID**. We need this data in order to ask OpenAI to submit our file as a batch request. Once we do though, OpenAI will give us a similar response, but now with a **Batch ID**. It is *extremely* cruicial that we keep these file and batch IDS accessible. This is because OpenAI's API functions as a sort of black box in the sense that once a submission has been made, there is no way to access it again unless you hang on to the API response that they provide when you make the submission. To address this concern, we save all of our File IDs and Batch IDs to a jsonl file which we can refer to later. It is very important that you have access to this file, as it is needed to collect the data after is has finished processing.

Because batch submissions are *asynchronous* the response needs to be collected separetly from the original batch submission. OpenAI states that it will try its best to complete a submission within 24 hours, so it should take at most a day to collect the results. We handle the upload file and submit batch step in one block of code, and the collection in a separate block.

The submission code below will take every batch submission file and upload it and then submit it as a batch request, and write the upload and batch submission responses to separate jsonl output files.

### Input for Upload & Batch Submission

- A directory containing prepared batch submission jsonl files
- A path & name to where you want to write the .jsonl output files to (shouldn't have output files currently in it)
- An API key from OpenAI (keep this a secret)
- Optionally, a metadata description for the batch submission

In [ ]:
pip install openai

In [ ]:
import os
import json
import glob
from openai import OpenAI

client = OpenAI(api_key='Your OpenAI API Key Here')

# Directory containing the .jsonl files you want to upload.
INPUT_DIRECTORY = "your/input/directory/here"

# Output file for storing upload responses (one JSON per line).
UPLOAD_RESPONSES_FILE = "your/.jsonl/upload/file/path/here"

# Output file for storing batch submission responses (one JSON per line).
BATCH_RESPONSES_FILE = "your/.jsonl/batch/file/path/here"

def uploadInputFiles():
    # Use glob to find all .jsonl files in the directory
    jsonl_files = glob.glob(os.path.join(INPUT_DIRECTORY, "*.jsonl"))

    for file_path in jsonl_files:
        with open(file_path, "rb") as f:
            # Create (upload) the file, returns a FileObject (not JSON serializable)
            upload_response = client.files.create(
                file=f,
                purpose="batch"
            )

            # Convert the FileObject to a JSON-serializable dict
            serializable_response = {
                "id": upload_response.id,
                "bytes": upload_response.bytes,
                "created_at": upload_response.created_at,
                "filename": upload_response.filename,
                "object": upload_response.object,
                "purpose": upload_response.purpose,
                "status": upload_response.status,
                "status_details": upload_response.status_details,
            }

        # Append the response (as JSON) on a new line in UPLOAD_RESPONSES_FILE
        with open(UPLOAD_RESPONSES_FILE, "a") as upload_out:
            upload_out.write(json.dumps(serializable_response) + "\n")

    print(f"Upload complete. Responses written to {UPLOAD_RESPONSES_FILE}")

def submitBatch():
    # Make sure the file exists and has content
    if not os.path.exists(UPLOAD_RESPONSES_FILE):
        print(f"No upload responses file found: {UPLOAD_RESPONSES_FILE}")
        return

    with open(UPLOAD_RESPONSES_FILE, "r") as upload_in:
        for line in upload_in:
            # Safely parse JSON; skip if empty
            line = line.strip()
            if not line:
                continue

            try:
                upload_response = json.loads(line)
            except json.JSONDecodeError:
                continue

            # Extract the file ID from the upload response
            file_id = upload_response.get("id")
            if not file_id:
                # If there's no 'id' field, skip
                continue

            # Submit a batch using that file_id, returns a Batch object
            batch_response = client.batches.create(
                input_file_id=file_id,
                endpoint="/v1/chat/completions",
                completion_window="24h",
                metadata={
                    "description": "Optionally, add a description"
                }
            )

            # Convert the Batch object to a JSON-serializable dict
            # Note that 'request_counts' is another custom object we break into a dict.
            batch_serializable = {
                "id": batch_response.id,
                "completion_window": batch_response.completion_window,
                "created_at": batch_response.created_at,
                "endpoint": batch_response.endpoint,
                "input_file_id": batch_response.input_file_id,
                "object": batch_response.object,
                "status": batch_response.status,
                "cancelled_at": batch_response.cancelled_at,
                "cancelling_at": batch_response.cancelling_at,
                "completed_at": batch_response.completed_at,
                "error_file_id": batch_response.error_file_id,
                "errors": batch_response.errors,
                "expired_at": batch_response.expired_at,
                "expires_at": batch_response.expires_at,
                "failed_at": batch_response.failed_at,
                "finalizing_at": batch_response.finalizing_at,
                "in_progress_at": batch_response.in_progress_at,
                "metadata": batch_response.metadata,
                "output_file_id": batch_response.output_file_id,
            }

            # If batch_response.request_counts is another object, break it down:
            if batch_response.request_counts:
                batch_serializable["request_counts"] = {
                    "completed": batch_response.request_counts.completed,
                    "failed": batch_response.request_counts.failed,
                    "total": batch_response.request_counts.total,
                }
            else:
                batch_serializable["request_counts"] = None

            # Write the batch response (as JSON) on a new line
            with open(BATCH_RESPONSES_FILE, "a") as batch_out:
                batch_out.write(json.dumps(batch_serializable) + "\n")

    print(f"Batch submission complete. Responses written to {BATCH_RESPONSES_FILE}")


# Example usage:
if __name__ == "__main__":

    # SAFETY CHECKS: If the output files already exist, abort.
    # This prevents overwriting and prevents ANY API calls.
    if os.path.exists(UPLOAD_RESPONSES_FILE):
        err_msg = (
            f"Error: The file '{UPLOAD_RESPONSES_FILE}' already exists. "
            "Refusing to overwrite. Aborting operation."
        )
        print(err_msg)
        raise FileExistsError(err_msg)

    if os.path.exists(BATCH_RESPONSES_FILE):
        err_msg = (
            f"Error: The file '{BATCH_RESPONSES_FILE}' already exists. "
            "Refusing to overwrite. Aborting operation."
        )
        print(err_msg)
        raise FileExistsError(err_msg)

    # If we get here, both files do NOT exist, so it's safe to proceed.
    uploadInputFiles()
    submitBatch()

## Collecting the Batch

Once the 24 hour window has passed, as long as there have been no errors, the submission is likely complete. So, we can use the files generated from the batch submission to collect our data from the AI model, and write it to a jsonl file. We will have as many output files as we did input, and will write these to a specified directory. Note that without the batch id files it is impossible to retreive the data!

### Necessary Elements

- OpenAI API key
- Path to batch id file
- Path to output directory

In [ ]:
import os
import json

from openai import OpenAI

client = OpenAI(api_key='Your API Key Here')

def collect_results_from_file():

    input_file_path = "path/to/your/batch/id/file"

    output_directory = "path/to/your/output/directory"

    with open(input_file_path, 'r', encoding='utf-8') as f:
        for line_number, line in enumerate(f, start=1):
            line = line.strip()
            if not line:
                continue  # Skip empty lines

            try:
                batch_info = json.loads(line)
            except json.JSONDecodeError:
                print(f"Line {line_number} is not valid JSON. Skipping.")
                continue
            
            # Extract the batch ID
            batch_id = batch_info.get("id")
            if not batch_id:
                print(f"Line {line_number} has no 'id' field. Skipping.")
                continue

            # Retrieve the batch
            try:
                batch_response = client.batches.retrieve(batch_id)
            except Exception as e:
                print(f"Error retrieving batch {batch_id}: {e}")
                continue

            # Check if there's an output_file_id
            output_file_id = batch_response.output_file_id
            if not output_file_id:
                print(f"No output file for batch {batch_id}, possibly still in progress.")
                continue

            # Retrieve the content of the output file
            try:
                file_response = client.files.content(output_file_id)
            except Exception as e:
                print(f"Error retrieving content for output_file_id {output_file_id}: {e}")
                continue

            # Create a unique output file name using the batch_id
            output_file_name = f"batch_{batch_id}.jsonl"
            output_file_path = os.path.join(output_directory, output_file_name)

            # Write the file content to disk
            try:
                with open(output_file_path, "w", encoding='utf-8') as output_file:
                    output_file.write(file_response.text)
                print(f"Saved output for batch {batch_id} to {output_file_path}")
            except Exception as e:
                print(f"Error writing to file {output_file_path}: {e}")
                continue

collect_results_from_file()

### Checking Batch Status

After making the batch submission, you may want to check the status of a batch submission. In order to do this, you will need a batch id to make the API call. After it has been made, you will get a response similar to when the batch submission was executed. The most important part of this response will be the status field. There are several possibilities for what this could be, but there are a few typical ones. "Validating" is the status provided when the initial batch submission is made. "in_progress" means that the submission is currently in the queue. "completed" means that the submission is done and ready for collection. "failed" means that something went wrong with the submission. Usually, the response will inform you of the error, but likely issues would be an invalid input format, exceeding the daily token rate limit, or exceeding 50000 lines per file or 200MB per file.

### Input
- OpenAI API key
- A single batch ID

In [ ]:
from openai import OpenAI

client = OpenAI(api_key='Your API Key Here')

retrieval = client.batches.retrieve("batchid_here")
print(retrieval)